In [1]:
import sys
#sys.path.insert(0, "../..")

from typing import Callable, Protocol, Dict, Optional, Iterator, List, Tuple

import gin
from pathlib import Path
import numpy as np
import math
from datetime import datetime

import torch
from torch import nn
import torch.optim as optim
from torch.optim import Optimizer
from torchinfo import summary
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#from src.data import make_dataset
#from src.models import metrics
#from src.models import train_model # om de functie train_loop binnen te halen

# ------------------------------------------------------------------------------- #
## Alle imports benodigd voor de functie train_loop uit train_model.py 

import tensorflow as tf  # noqa: F401

# needed to make summarywriter load without error
from loguru import logger
from numpy import Inf
from ray import tune


from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from tqdm import tqdm

#from src.models.metrics import Metric
#from src.typehinting import GenericModel
#from src.data import data_tools

2022-12-17 17:55:26.969653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-17 17:55:28.051463: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-17 17:55:28.051567: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-17 17:55:28.051577: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
## benodigde functie uitmake_dataset.py

#@gin.configurable
def get_MNIST(  # noqa: N802
    data_dir: Path, batch_size: int
) -> Tuple[DataLoader, DataLoader]:

    training_data = datasets.FashionMNIST(
        root=data_dir,
        train=True,
        download=True,
        transform=ToTensor(),
    )

    test_data = datasets.FashionMNIST(
        root=data_dir,
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader

In [3]:
## Benodigde funties uit metrics.py

Tensor = torch.Tensor

class Metric:
    def __repr__(self) -> str:
        raise NotImplementedError

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        raise NotImplementedError

class Accuracy(Metric):
    def __repr__(self) -> str:
        return "Accuracy"

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        return (yhat.argmax(dim=1) == y).sum() / len(yhat)


In [4]:
class GenericModel(Protocol):
    train: Callable
    eval: Callable
    parameters: Callable

    def __call__(self, *args, **kwargs) -> torch.Tensor:
        pass


In [5]:
def count_parameters(model: GenericModel) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [6]:
## uit data_tools.py

def dir_add_timestamp(log_dir: Optional[Path] = None) -> Path:
    if log_dir is None:
        log_dir = Path(".")
    log_dir = Path(log_dir)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M")
    log_dir = log_dir / timestamp
    logger.info(f"Logging to {log_dir}")
    if not log_dir.exists():
        log_dir.mkdir(parents=True)
    return log_dir

In [7]:
## Definieren functie train_loop

#@gin.configurable
def trainloop(
    epochs: int,
    model: GenericModel,
    optimizer: torch.optim.Optimizer,
    learning_rate: float,
    loss_fn: Callable,
    metrics: List[Metric],
    train_dataloader: Iterator,
    test_dataloader: Iterator,
    log_dir: Path,
    train_steps: int,
    eval_steps: int,
    patience: int = 10,
    factor: float = 0.9,
    tunewriter: bool = False,
) -> GenericModel:
    
    optimizer_: torch.optim.Optimizer = optimizer(
        model.parameters(), lr=learning_rate
    )  # type: ignore

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_,
        factor=factor,
        patience=patience,
    )

    if not tunewriter:
        log_dir = dir_add_timestamp(log_dir)
        writer = SummaryWriter(log_dir=log_dir)
        #write_gin(log_dir, gin.config_str())

        images, _ = next(iter(train_dataloader))
        if len(images.shape) == 4:
            grid = make_grid(images)
            writer.add_image("images", grid)
        writer.add_graph(model, images)

    for epoch in tqdm(range(epochs)):
        train_loss = trainbatches(
            model, train_dataloader, loss_fn, optimizer_, train_steps
        )

        metric_dict, test_loss = evalbatches(
            model, test_dataloader, loss_fn, metrics, eval_steps
        )

        scheduler.step(test_loss)

        if tunewriter:
            tune.report(
                iterations=epoch,
                train_loss=train_loss,
                test_loss=test_loss,
                **metric_dict,
            )
        else:
            writer.add_scalar("Loss/train", train_loss, epoch)
            writer.add_scalar("Loss/test", test_loss, epoch)
            for m in metric_dict:
                writer.add_scalar(f"metric/{m}", metric_dict[m], epoch)
            lr = [group["lr"] for group in optimizer_.param_groups][0]
            writer.add_scalar("learning_rate", lr, epoch)
            metric_scores = [f"{v:.4f}" for v in metric_dict.values()]
            logger.info(
                f"Epoch {epoch} train {train_loss:.4f} test {test_loss:.4f} metric {metric_scores}"  # noqa E501
            )

    return model

In [8]:
## uit train_model.py

def trainbatches(
    model: GenericModel,
    traindatastreamer: Iterator,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    train_steps: int,
) -> float:
    model.train()
    train_loss: float = 0.0
    for _ in tqdm(range(train_steps)):
        x, y = next(iter(traindatastreamer))
        optimizer.zero_grad()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.detach().numpy()
    train_loss /= train_steps
    return train_loss

In [9]:
## uit train_model.py

def evalbatches(
    model: GenericModel,
    testdatastreamer: Iterator,
    loss_fn: Callable,
    metrics: List[Metric],
    eval_steps: int,
) -> Tuple[Dict[str, float], float]:
    model.eval()
    test_loss: float = 0.0
    metric_dict: Dict[str, float] = {}
    for _ in range(eval_steps):
        x, y = next(iter(testdatastreamer))
        yhat = model(x)
        test_loss += loss_fn(yhat, y).detach().numpy()
        for m in metrics:
            metric_dict[str(m)] = (
                metric_dict.get(str(m), 0.0) + m(y, yhat).detach().numpy()
            )

    test_loss /= eval_steps
    for key in metric_dict:
        metric_dict[str(key)] = metric_dict[str(key)] / eval_steps
    return metric_dict, test_loss


In [ ]:
#gin.parse_config_file("model.gin")

In [ ]:
#sys.path

In [10]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cpu device


In [14]:
## Model 1: CNN_een met een convolutional layer & drie linear layers

#@gin.configurable
class CNN_een_drie(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(6272, 3136),
            nn.ReLU(),
            nn.Linear(3136, 1568),
            nn.ReLU(),
            nn.Linear(1568, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_een_drie = CNN_een_drie(10, 3, 32, 32).to(device)

In [15]:
## Model 2.2: CNN_twee met twee convolutional layers & twee linear layers

#@gin.configurable
class CNN_twee_twee(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_twee = CNN_twee_twee(10, 3, 32, 32).to(device)

In [16]:
## Model 2.3: CNN_twee met twee convolutional layers en drie linear layers

#@gin.configurable
class CNN_twee_drie(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_drie = CNN_twee_drie(10, 3, 32, 32).to(device)

In [17]:
## Model 2.4: CNN_twee met twee convolutional layers & vier linear layers

#@gin.configurable
class CNN_twee_vier(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, 144),
            nn.ReLU(),
            nn.Linear(144, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_twee_vier = CNN_twee_vier(10, 3, 32, 32).to(device)

In [18]:
## Model 3.2: CNN_drie met twee convolutional layers & twee linear layers

#@gin.configurable
class CNN_drie_twee(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_drie_twee = CNN_drie_twee(10, 3, 32, 32).to(device)

In [19]:
## Model 3.3: CNN_drie met drie convolutional layers

#@gin.configurable
class CNN_drie_drie(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_drie_drie = CNN_drie_drie(10, 3, 32, 32).to(device)

In [20]:
## Model 3.4: CNN_drie met drie convolutional layers & vier linear layers

#@gin.configurable
class CNN_drie_vier(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# a = aantal classes, b = kernel_size, c = grootte filter 1, d = grootte  filter2
model_drie_vier = CNN_drie_vier(10, 3, 32, 32).to(device)

In [21]:
modellen = [
            model_een_drie,
            model_twee_twee,
            model_twee_drie,
            model_twee_vier,
            model_drie_twee,
            model_drie_drie,
            model_drie_vier,
            ]


In [ ]:
for model in modellen:
    print(summary(model, input_size=(64, 1, 28, 28)))

In [ ]:
gin.parse_config_file("model.gin")

In [12]:
datadir =  Path("/home/admindme/code/ML22_opdracht1/data/raw/FashionMNIST")
train_dataloader, test_dataloader = get_MNIST(datadir, 64)
len(train_dataloader), len(test_dataloader)

(938, 157)

In [28]:
model = model_drie_drie
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = Accuracy()

model = trainloop(
            epochs=epochs,
            model=model,
            optimizer=torch.optim.Adam,
            learning_rate= 0.001,
            loss_fn=loss_fn,
            metrics=[accuracy],
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            log_dir="../../models/test/",
            train_steps=len(train_dataloader),
            eval_steps=150,
            #patience= ,
            #factor= ,
            #tunewriter= ,
            )

2022-12-17 18:41:34.656 | INFO     | __main__:dir_add_timestamp:9 - Logging to ../../models/test/20221217-1841
100%|██████████| 938/938 [00:29<00:00, 31.95it/s]
2022-12-17 18:42:06.491 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.7348 test 0.5453 metric ['0.7887']
100%|██████████| 938/938 [00:30<00:00, 30.80it/s]
2022-12-17 18:42:39.371 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4631 test 0.4382 metric ['0.8363']
100%|██████████| 938/938 [00:30<00:00, 31.05it/s]
2022-12-17 18:43:11.918 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3735 test 0.3568 metric ['0.8680']
100%|██████████| 938/938 [00:31<00:00, 29.95it/s]
2022-12-17 18:43:45.635 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3258 test 0.3278 metric ['0.8862']
100%|██████████| 938/938 [00:31<00:00, 29.69it/s]
2022-12-17 18:44:19.612 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3058 test 0.3068 metric ['0.8888']
100%|██████████| 938/938 [00:30<00:00, 30.97it/s]
2022-12-17 18:44:52.236 | INF